In [1]:
import pandas as pd
import numpy as np

In [2]:
root_path = '/mnt/data/vikuen/data/guardian/'

In [3]:
train_df = pd.read_csv(root_path + 'train-set_all.csv')

In [4]:
authors_path = root_path + 'train_author_ids.npy'

In [5]:
author_ids = np.load(authors_path)

In [6]:
len(author_ids)

95617

In [7]:
num_comments = train_df.groupby('author_id')['comment_id'].nunique().reset_index()

In [8]:
author_ids = num_comments[num_comments['comment_id'] > 2]['author_id'].unique()

In [9]:
train_df_selected = train_df[train_df['author_id'].isin(author_ids)].reset_index()

In [10]:
train_df_selected['timestamp'] = pd.to_datetime(train_df_selected['timestamp'])

In [11]:
train_df_selected = train_df_selected.sort_values(by='timestamp')

In [12]:
to_represent_comment_ids = []
def get_representation_data(x):
    length = len(x)
    select = length // 2
    if select > 20:
        select = 20
        
    elements = x.head(select)
    for el in elements.tolist():
        to_represent_comment_ids.append(el)
    return select
    

In [13]:
out = train_df_selected.groupby('author_id')['comment_id'].apply(get_representation_data)

In [14]:
out.reset_index().describe()

,author_id,comment_id
count,7.654300e+04,76543.000000
mean,1.850604e+05,8.444417
std,2.042181e+05,7.401941
min,6.000000e+00,1.000000
25%,3.981400e+04,2.000000
50%,1.187150e+05,5.000000
75%,2.474485e+05,17.000000
max,1.245769e+06,20.000000


In [15]:
len(to_represent_comment_ids)

646361

In [16]:
train_df_selected = train_df_selected[~train_df_selected['comment_id'].isin(np.array(to_represent_comment_ids))]

In [17]:
len(train_df_selected)

3071451

In [18]:
np.save('/mnt/data/vikuen/data/guardian/comment_ids_to_train.npy', train_df_selected['comment_id'].unique())

In [20]:
np.save('/mnt/data/vikuen/data/guardian/comment_ids_not_to_train.npy', np.array(to_represent_comment_ids))

In [15]:
train_df_selected.groupby('author_id')['comment_id'].nunique().describe()

count    76543.000000
mean        48.571548
std        140.669069
min          3.000000
25%          5.000000
50%         10.000000
75%         34.000000
max       6215.000000
Name: comment_id, dtype: float64